In [1]:
import os
import re
import ast
import pandas as pd
from openpyxl import load_workbook
import time

time_per_file = []

# path and file_name
def read_file(path_file):

    dataframe = pd.read_excel(path_file)
    return dataframe
#     else:
#         dataframe = pd.read_excel(path_file, sheet_name="Application Template")
#         return pd.DataFrame()

#Drop unnecessary columns
def drop_columns(df):

    drop_column_names = []
    
    for col in dataframe.columns:
        if re.search("^Unnamed", col, re.IGNORECASE) is None:
            drop_column_names.append(col)
    
    df = df.drop(drop_column_names, axis=1)

    new_columns = []
    for col in df.columns:
        col = col.replace(".","_")
        col = col.replace(": ","_")
        new_columns.append(col.lower())
    
    df.columns = new_columns    
    return df


In [2]:
def match_related_columns(df):
    df["row_number"] = df.index # assigning rows for individual's grouping of data later
    df["unnamed_1"] = df["unnamed_1"].replace(" ","_",regex=True).str.lower() #converting this to variable-like
    df["unnamed_2"] = df["unnamed_2"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN" else None).astype(str)

    df["unnamed_4"] = df["unnamed_4"].replace(" ","_",regex=True).str.lower()
    df["unnamed_5"] = df["unnamed_5"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN"else None).astype(str)

    df["unnamed_6"] = df["unnamed_6"].replace(" ","_",regex=True).str.lower()
    df["unnamed_7"] = df["unnamed_7"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN"else None).astype(str)

    df["unnamed_9"] = df["unnamed_9"].replace(" ","_",regex=True).str.lower()
    df["unnamed_10"] = df["unnamed_10"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN"else None).astype(str) 

    df["unnamed_20"] = df["unnamed_20"].replace(" ","_",regex=True).str.lower()
    df["unnamed_21"] = df["unnamed_21"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN"else None).astype(str)

    #combining related data - by category and value
    df["Notice_Credit_Decision"] = df["unnamed_1"].astype(str) +" | " +df["unnamed_2"].astype(str)
    df["Basic_Info"] = df["unnamed_4"].astype(str) +" | " +df["unnamed_5"].astype(str)
    df["MC_Details"] = df["unnamed_6"].astype(str) +" | " +df["unnamed_7"].astype(str)
    df["Credit_Info"] = df["unnamed_9"].astype(str) +" | " +df["unnamed_10"].astype(str)
    df["DTIR"] = df["unnamed_20"].astype(str) +" | " +df["unnamed_21"].astype(str)
    
#     for col in df.columns:
#         df[col] = df[col].astype(str)
#         df[col] = df[col].str.replace(",","")
    
    groupings_list = [df["Notice_Credit_Decision"].to_dict(), df["Basic_Info"].to_dict(), df["MC_Details"].to_dict(), df["Credit_Info"].to_dict(), df["DTIR"].to_dict()]

# all_data = []
    return groupings_list


def group_dataset(x):
# def group_dataset(x, dataset, dataset_list):
#     x is a multiple of 26
    y = x/26
    
    applicant_data_set = []
    for group in grouping:
        applicant_data={}
        applicant_set_per_group = {k:v for k,v in group.items() if k>=x and k<26*(y+1)}
        applicant_data[x] = {v for v in applicant_set_per_group.values()}
        
        applicant_data_set.append(applicant_data)

    for row_number in applicant_data_set[0].keys():
        combined_data_tuple = []
        for d in applicant_data_set:
            combined_data_tuple.append(d[row_number])
    
    #manipulate combined_data_tuplee
    t = []
    for s in combined_data_tuple:
        s = list(s)
        for r in s:
            t.append(r)
            
    all_data.append(t)

def get_first_occurence(dataset):
    for key, value in dataset[0].items():
        if key%26 == 0:
            group_dataset(key)
            



In [3]:
# for x in all_data_list
def generate_columns(data):
    df = pd.DataFrame(data)
    df["row_number"] = df.index 
    
    for col in df.columns:
        if col!="row_number":
            feature_columns  = [ "{col}_category".format(col=col), "{col}_data".format(col=col) ]
            splitted_data = df[col].str.split(" | ", n = 2, expand = True) 
            df[feature_columns[0]]= splitted_data[0]
            df[feature_columns[1]]= splitted_data[2]
            df["category"] = splitted_data[0]
            df["data"] = splitted_data[2]

            if col == 0:
                df_category_data = df[["row_number","0_category" ,"0_data"]]
                df_category_data.columns = ["row_number", "category", "data"]
            else:
                df_category_data = pd.concat([df_category_data[["row_number","category", "data"]], df[["row_number","category", "data"]]])

            df = df.drop(col, axis=1)

    df_category_data = df_category_data.mask(df_category_data["category"].eq('None'))
    df_category_data = df_category_data.mask(df_category_data["category"].eq('nan'))
    df_category_data = df_category_data.mask(df_category_data["category"].eq(' '))
    df_category_data = df_category_data.mask(df_category_data["category"].eq('_'))
    
    df_category_data = df_category_data.mask(df_category_data["data"].eq('None'))
    df_category_data = df_category_data.mask(df_category_data["data"].eq('nan'))
    df_category_data = df_category_data.mask(df_category_data["data"].eq(' '))
    df_category_data = df_category_data.mask(df_category_data["data"].eq('_'))

    df_category_data.drop_duplicates(subset=["row_number","category"], keep="first", inplace=True)
    df_category_data.reset_index().set_index(['row_number'])

    #this is where we generate columns for our excel file
    df_generated_columns = df_category_data.pivot(index="row_number", columns=["category"], values="data")

    df_generated_columns = df_generated_columns.dropna(how="all", axis=0)
    df_generated_columns = df_generated_columns.dropna(how="all", axis=1)

    return df_generated_columns

# number application_date
# 0   10/07/2020

def generate_file(path, yy, mm, file_name, df):
    #positioning important fields
    df = df.dropna(subset=["customer_name"], axis=0)
    customer_name = df.pop("customer_name")
    age = df.pop("age")
    gross_income =df.pop("gross_income")
    dtir = df.pop("dtir")
    df.insert(0, "customer_name", customer_name)
    df.insert(1, "age", age)
    df.insert(2, "gross_income", gross_income)
    df.insert(3, "dtir", dtir)
    file_name = file_name.replace(" ","_").lower()
    complete_file_name = "{}{}/{}/Cleaned_{}".format(path,yy,mm,file_name)
    print(complete_file_name)
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists("{}{}".format(path,yy)):
        os.makedirs("{}{}".format(path,yy))
    if not os.path.exists("{}{}/{}/".format(path,yy,mm)):
        os.makedirs("{}{}/{}/".format(path,yy,mm))
        
    if not os.path.isfile(complete_file_name):
        df.to_excel(complete_file_name)
    
    
if __name__=='__main__':
    path = '../Sample/'
    path_result = '../Result/'
    years = os.listdir(path)
    
    for year in years:
        year_path = "{}{}/".format(path,year)
        months = os.listdir(year_path)
        if year != "2018":
            for month in months:
                
                month_path = "{}{}".format(year_path, month)
                print(month_path)
                files = os.listdir(month_path)
                
                for file in files:
                    
                    file_path = "{}/{}".format(month_path, file)
                    print(file_path)
                    if os.path.isfile(file_path):
                        start_time = time.time()
                        dataframe = read_file(file_path)
                        dataframe = drop_columns(dataframe)
                        grouping = match_related_columns(dataframe) #returns a list of dictionaries [{}, {}, {}]
                        all_data = []
                        get_first_occurence(grouping)
                        tabular_data = generate_columns(all_data)
                        generate_file(path_result, year, month, file, tabular_data)
                        end_time = time.time() - start_time
                        
                        time_per_file.append(time.strftime("%H:%M:%S",time.gmtime(end_time)))
                        
                        f = open("time_november_2019_onwards.txt", "w")
                        f.write(', '.join(map(str, time_per_file)))
                        f.close()
    
    
    
                        
        


../Sample/2019/April_xlsx
../Sample/2019/April_xlsx/APPLICATION TEMPLATE 2019 April joyce.xlsx
../Result/2019/April_xlsx/Cleaned_application_template_2019_april_joyce.xlsx
../Sample/2019/April_xlsx/APPLICATION TEMPLATE April  2019 chelle.xlsx
../Result/2019/April_xlsx/Cleaned_application_template_april__2019_chelle.xlsx
../Sample/2019/April_xlsx/APPLICATION TEMPLATE APRIL 2019 APRIL.xlsx
../Result/2019/April_xlsx/Cleaned_application_template_april_2019_april.xlsx
../Sample/2019/April_xlsx/Application Templates 2019 April.xlsx
../Result/2019/April_xlsx/Cleaned_application_templates_2019_april.xlsx
../Sample/2019/April_xlsx/Application Templates April 2019 fnb.xlsx
../Result/2019/April_xlsx/Cleaned_application_templates_april_2019_fnb.xlsx
../Sample/2019/April_xlsx/Application Templates APRIL 2019. - TINA.xlsx
../Result/2019/April_xlsx/Cleaned_application_templates_april_2019._-_tina.xlsx
../Sample/2019/April_xlsx/Lloyd Application Templates 2019.xlsx
../Result/2019/April_xlsx/Cleaned_ll

../Result/2019/March_xlsx/Cleaned_application_temlate_march_2019_manuel.xlsx
../Sample/2019/March_xlsx/Application Templates 2019 Joyce.xlsx
../Result/2019/March_xlsx/Cleaned_application_templates_2019_joyce.xlsx
../Sample/2019/March_xlsx/Application Templates March 2019  chelle.xlsx
../Result/2019/March_xlsx/Cleaned_application_templates_march_2019__chelle.xlsx
../Sample/2019/March_xlsx/Application Templates MARCH 2019 - TINA.xlsx
../Result/2019/March_xlsx/Cleaned_application_templates_march_2019_-_tina.xlsx
../Sample/2019/March_xlsx/Application Templates March 2019 Desktop (Autosaved).xlsx
../Result/2019/March_xlsx/Cleaned_application_templates_march_2019_desktop_(autosaved).xlsx
../Sample/2019/March_xlsx/Application Templates March 2019 fnb.xlsx
../Result/2019/March_xlsx/Cleaned_application_templates_march_2019_fnb.xlsx
../Sample/2019/March_xlsx/lloyd march Application Templates 2019 update.xlsx
../Result/2019/March_xlsx/Cleaned_lloyd_march_application_templates_2019_update.xlsx
../

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b',DESKTOP'